In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import plotly.express as px
import json
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

## 数据读取

In [3]:
json_io=r"geojson-map-china\china.json"
gs_data = open(json_io, encoding='utf8').read()
gs_data = json.loads(gs_data)
#整理plotly需要的格式：
for i in range(len(gs_data["features"])):
    gs_data["features"][i]["id"]=gs_data["features"][i]["properties"]["id"]#id前置
    gs_data["features"][i]["name"]=gs_data["features"][i]["properties"]["name"]
### 匹配id和区域
geo_id=[]
geo_name=[]
for i in range(len(gs_data["features"])):
    geo_id.append(gs_data["features"][i]["id"])
    geo_name.append(gs_data["features"][i]['properties']["name"])
geo_data=pd.DataFrame({"id":geo_id,"区域":geo_name})
data = pd.read_excel("城投债数据_t.xlsx")
GK=pd.read_excel("Credit_Assistant.xlsx",sheet_name="国开可比基准",skiprows=1,index_col=0).iloc[2:,:]
GK_yield_base=GK.tail(1).T
GK_yield_base.columns=["GK_yield"]
GK_yield_base["期限"]=[1,2,3,4,5]


## 函数

In [6]:

## 定义根据债券余额加权的点乘积：
def weighted_premium(dff_VS_GK):
    weighted_premium=np.dot(dff_VS_GK["券种利差"],dff_VS_GK["债券余额\n[日期] 最新\n[单位] 亿"]/dff_VS_GK["债券余额\n[日期] 最新\n[单位] 亿"].sum())
    return round(weighted_premium,2)

def get_credit_premium():
    data= pd.read_excel("Credit_Assistant.xlsx",skiprows=1,index_col=0).iloc[2:,:]
    index_code=pd.read_excel("Credit_Assistant.xlsx",skiprows=1,index_col=0).iloc[1,:].tolist()
    index_name=pd.read_excel("Credit_Assistant.xlsx").iloc[0,1:].tolist()
    str=","
    err,df=w.edb(str.join(index_code),"2019-01-01", dt.datetime.today().strftime("%Y-%m-%d"),"Fill=Previous",usedf=True)
    df.columns=index_name
    return df

def get_credit_vs_gk_data():
    GK_yield_base=GK_updated_yield()
    dff_VS_GK=pd.merge(df[df["期限"]<5],GK_yield_base,left_on=["期限_匹配"],right_on=["期限"],how="left")
    dff_VS_GK["券种利差"]=(dff_VS_GK["债券估值(YY)\n[单位] %"]-dff_VS_GK["GK_yield"])*100
    dff_VS_GK=dff_VS_GK[dff_VS_GK["券种利差"].isna()==False]
    return dff_VS_GK

info_dimension="券种利差","债券余额\n[日期] 最新\n[单位] 亿"

def province_credit_premium_fig(df):

    dff=pd.merge(pd.DataFrame(df,columns=["信用利差"]),geo_data,left_on="区域",right_on="区域")
    fig = px.choropleth_mapbox(dff, geojson=gs_data, locations='id', color='信用利差',
            range_color=(20, 400),
            zoom=3, center = {"lat": 37.4189, "lon": 116.4219},
            mapbox_style='carto-positron',
            hover_data=["区域", "信用利差"]
            )

    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


In [7]:
## 筛选非PPN
data = data[data["证券简称"].str.contains("PPN")==False]
#确定债券的可比期限
data["含权债行权期限"]=data["含权债行权期限"].fillna(10)
data["含权债行权期限"]=data["含权债行权期限"]*365
df=pd.DataFrame(data=data)
df["期限"]=((data[["含权债行权期限","剩余期限(天)\n[日期] 最新\n[单位] 天"]].min(axis=1))/365).round(2)
df["期限_匹配"]=((data[["含权债行权期限","剩余期限(天)\n[日期] 最新\n[单位] 天"]].min(axis=1))/365).round(0)
dff_VS_GK=pd.merge(df[df["期限"]<5],GK_yield_base,left_on=["期限_匹配"],right_on=["期限"],how="left")
dff_VS_GK["券种利差"]=(dff_VS_GK["债券估值(YY)\n[单位] %"]-dff_VS_GK["GK_yield"])*100
dff_VS_GK=dff_VS_GK[dff_VS_GK["券种利差"].isna()==False]
info_dimension="券种利差","债券余额\n[日期] 最新\n[单位] 亿"
province_credit_premium=dff_VS_GK.groupby("区域")[info_dimension].apply(lambda x : weighted_premium(x))

In [8]:
# 每个省份对应的城市
province_city = []
for name,group in dff_VS_GK.groupby("区域")['城市']:
    temp = dict()
    temp['label'] = name
    temp['value'] = group.unique().tolist()
    province_city.append(temp)
    
available_cities = dff_VS_GK['城市'].unique()


    
    
    

In [6]:
fig = province_credit_premium_fig(province_credit_premium)
info_dimension="券种利差","债券余额\n[日期] 最新\n[单位] 亿"
app.layout = html.Div(
    
    [
        html.Div(
        [dcc.Graph(id='China-bond-map',
              figure = fig),
         dcc.Graph( id='bond-by-city')]
        ,style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
        
        dcc.Dropdown(id = 'choose-of-cities',
                  options = [{'label': i, 'value': i} for i in available_cities],
                  value=['上海市','北京市'],
                  placeholder="请选择选择想要比较的城市",
                  multi = True),

        dcc.Graph(id = 'compare-bond-by-city')
    ]
    
)
    
    


@app.callback(
    dash.dependencies.Output('bond-by-city', 'figure'),
    [dash.dependencies.Input('China-bond-map', 'clickData')])
def update_figure(clickData):
    df_province = dff_VS_GK[dff_VS_GK['区域'] == clickData['区域']].groupby("城市")["券种利差","债券余额\n[日期] 最新\n[单位] 亿"].apply(lambda x : weighted_premium(x))
    dff = pd.DataFrame(df_province,columns = ['信用利差']).reset_index()
    fig = px.bar(dff, x="城市", y="信用利差")

 #   fig.update_layout(transition_duration=500)

    return fig

@app.callback(
    dash.dependencies.Output('compare-bond-by-city', 'figure'),
    [dash.dependencies.Input('choose-of-cities', 'value')])
def compare_figure(cities):
    dff = dff_VS_GK[dff_VS_GK['城市'].isin(cities)]["券种利差","债券余额\n[日期] 最新\n[单位] 亿"].apply(lambda x : weighted_premium(x))
    dff2 = pd.DataFrame(dff,columns = ['信用利差']).reset_index()
    fig = px.bar(dff2, x="城市", y="信用利差")

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [4]:
available_cities

NameError: name 'available_cities' is not defined

array(['南昌市', '武汉市', '西安市', '北京市', '宁波市', '南宁市', '郑州市', '天津市', '南京市',
       '沈阳市', '上海市', '成都市', '泸州市', '广州市', '青岛市', '莆田市', '重庆市', '济宁市',
       '珠海市', '绍兴市', '靖江市', '苏州市', '北海市', '长沙市', '淮安市', '湖州市', '济南市',
       '乌鲁木齐市', '海宁市', '常州市', '淮北市', '威海市', '宁德市', '贵阳市', '黔东南苗族侗族自治州',
       '杭州市', '潍坊市', '可克达拉市', '咸阳市', '太原市', '余姚市', '漳州市', '锦州市', '江阴市',
       '庄河市', '唐山市', '兰州市', '无锡市', '柳州市', '景德镇市', '长春市', '株洲市', '南通市',
       '江门市', '昆明市', '郴州市', '连云港市', '舟山市', '绵阳市', '扬州市', '盐城市', '徐州市',
       '赣州市', '吉安市', '北屯市', '衡阳市', '阿拉尔市', '泰州市', '湛江市', '宿迁市', '攀枝花市',
       '海口市', '滁州市', '抚州市', '厦门市', '石家庄市', '呼和浩特市', '张家口市', '上饶市', '佛山市',
       '巴中市', '开封市', '温州市', '嘉兴市', '遵义市', '日照市', '内江市', '德兴市', '襄阳市',
       '启东市', '荆州市', '醴陵市', '浏阳市', '芜湖市', '雅安市', '太仓市', '永安市', '黄冈市',
       '腾冲市', '建德市', '自贡市', '马鞍山市', '宜昌市', '眉山市', '遂宁市', '仁怀市', '周口市',
       '张家港市', '营口市', '阳江市', '哈尔滨市', '简阳市', '怀化市', '德州市', '安阳市', '吉林市',
       '延安市', '海安市', '诸暨市', '常熟市', '沧州市', '龙岩市', '松原市', '岳阳市', '平度市',
     

In [14]:
geo_data[geo_data["区域"]=="江苏省"]

,id,区域
24,32,江苏省


In [17]:
province_credit_premium_fig(df)

KeyError: '区域'

In [36]:
province_credit_premium_fig(province_credit_premium).layout

Layout({
    'coloraxis': {'cmax': 400,
                  'cmin': 20,
                  'colorbar': {'title': {'text': '信用利差'}},
                  'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                 '#46039f'], [0.2222222222222222, '#7201a8'],
                                 [0.3333333333333333, '#9c179e'],
                                 [0.4444444444444444, '#bd3786'],
                                 [0.5555555555555556, '#d8576b'],
                                 [0.6666666666666666, '#ed7953'],
                                 [0.7777777777777778, '#fb9f3a'],
                                 [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']]},
    'geo': {'fitbounds': 'locations', 'visible': True},
    'legend': {'tracegroupgap': 0},
    'mapbox': {'center': {'lat': 37.4189, 'lon': 116.4219},
               'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
               'style': 'carto-positron',
               'zoom': 3},
    'margin': {'b': 0, 

In [22]:
import do.data_organize as do

In [33]:
geo_data

,id,区域
0,65,新疆维吾尔自治区
1,54,西藏自治区
2,15,内蒙古自治区
3,63,青海省
4,51,四川省
5,23,黑龙江省
6,62,甘肃省
7,53,云南省
8,45,广西壮族自治区
9,43,湖南省


In [12]:
b=str(A)

In [13]:
import json
json.load(b)

AttributeError: 'str' object has no attribute 'read'

"{'points': [{'curveNumber': 0, 'pointNumber': 14, 'pointIndex': 14, 'location': '41', 'z': 166.49, 'customdata': '河南省'}]}"

In [19]:
a={"points": [{"curveNumber": 0, "pointNumber": 8, "pointIndex": 8, "location": "44", "z": 71.54, "customdata": "广东省"}]}

In [20]:
b=str(a)

In [25]:
b

"{'points': [{'curveNumber': 0, 'pointNumber': 8, 'pointIndex': 8, 'location': '44', 'z': 71.54, 'customdata': '广东省'}]}"

In [45]:
a= "{'curveNumber': 0, 'pointNumber': 8, 'pointIndex': 8, 'location': '44', 'z': 71.54, 'customdata': '广东省'}"

In [43]:
 import demjson


In [48]:
 v = demjson.decode(a)

In [50]:
a="{'points': [{'curveNumber': 0, 'pointNumber': 8, 'pointIndex': 8, 'location': '44', 'z': 71.54, 'customdata': '广东省'}]}"

In [51]:
v = demjson.decode(a)

In [55]:
v["points"][0][]

[{'curveNumber': 0,
  'pointNumber': 8,
  'pointIndex': 8,
  'location': '44',
  'z': 71.54,
  'customdata': '广东省'}]